# 모델 아키텍처

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import time
from tqdm import tqdm

In [31]:
# 1. 모델 아키텍처
class NewsSimilarityModel(nn.Module):
    """
    뉴스 유사도 계산 모델
    
    구조:
    1. 뉴스 A, B → Emb 1 (일단은 KLUE-BERT) → Context Vector (768차원)
    2. CV + 외부피처 → FCL (각각 별도) → Linear → 최종 벡터
    3. 두 최종 벡터 간 Cosine Similarity 계산
    """
    def __init__(self, 
                 embedding_model_name='klue/bert-base', # 학습 데이터셋 구성할 때의 임베딩 모델이랑 동일하게 설정해야 할 수 있음
                 embedding_dim=768, # BERT 임베딩 차원
                 external_feature_dim=10, # 외부 피처 차원(일단 10개로 넣어 둠)
                 fcl_hidden_dim=512, # FCL의 히든 레이어 차원
                 linear_output_dim=256): # 최종 출력 벡터 차원
        super().__init__()
        
        # 사전 학습 임베딩 모델 로드
        self.embedding_model = AutoModel.from_pretrained(embedding_model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
        
        # 프리징 - 임베딩 모델의 파라미터는 학습하지 않음
        for name, param in self.embedding_model.named_parameters():
            param.requires_grad = False # 임베딩 모델의 파라미터 고정

        # 텍스트 임베딩 차원과 외부 피처 차원을 합쳐서 입력 차원 계산
        input_dim = embedding_dim + external_feature_dim

        # 현재 뉴스(기준)에 대한 FCL과 선형 레이어
        self.fcl_current = nn.Sequential(
            nn.Linear(input_dim, fcl_hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(fcl_hidden_dim, fcl_hidden_dim)
        )

        # 과거 뉴스(비교 대상)에 대한 FCL과 선형 레이어
        self.fcl_past = nn.Sequential(
            nn.Linear(input_dim, fcl_hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(fcl_hidden_dim, fcl_hidden_dim)
        )
        
        # 최종 출력 벡터를 위한 선형 레이어
        self.linear_current = nn.Linear(fcl_hidden_dim, linear_output_dim)
        self.linear_past = nn.Linear(fcl_hidden_dim, linear_output_dim)

    def get_text_embedding(self, texts, device):
        """
        입력된 텍스트 리스트를 BERT 임베딩으로 변환
        → [CLS] 토큰의 출력만 사용
        """
        inputs = self.tokenizer(
            texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=512
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad(): # 프리징된 모델이므로 파라미터는 업데이트하지 않음
            outputs = self.embedding_model(**inputs)
            return outputs.last_hidden_state[:, 0, :] # [CLS] 토큰 출력 (batch_size, 768)

    def forward(self, news_a, news_b, ext_a, ext_b, device):
        """
        기준 뉴스 A와 비교 뉴스 B, 외부 피처들을 받아 각각의 최종 벡터를 반환
        """
        # 뉴스 본문 임베딩
        emb_a = self.get_text_embedding(news_a, device)
        emb_b = self.get_text_embedding(news_b, device)

        # 외부 피처와 concat → [text_embedding + external_feature]
        combined_a = torch.cat([emb_a, ext_a.to(device)], dim=1)
        combined_b = torch.cat([emb_b, ext_b.to(device)], dim=1)

        # 각각 FCL → Linear 통과시켜서 최종 벡터 추출
        current_vec = self.linear_current(self.fcl_current(combined_a))
        past_vec = self.linear_past(self.fcl_past(combined_b))
        return current_vec, past_vec


# 2. 손실 함수
class SimilarityLoss(nn.Module):
    def __init__(self, margin=0.5):
        super().__init__()
        self.margin = margin

    def forward(self, vec_a, vec_b, labels):
        """
        벡터 A, B 간의 코사인 유사도를 기반으로 손실 계산
        - 유사 뉴스(label=1): 유사도는 1에 가까워야 하므로 (1 - sim)^2
        - 비유사 뉴스(label=0): sim이 margin보다 크면 페널티 부여
        """
        cosine_sim = F.cosine_similarity(vec_a, vec_b, dim=1)

        # 양성 샘플 손실: 1과 가까울수록 손실이 작음
        pos_loss = labels * torch.pow(1 - cosine_sim, 2)

        # 음성 샘플 손실: margin보다 크면 손실 발생
        neg_loss = (1 - labels) * torch.pow(torch.clamp(cosine_sim - self.margin, min=0), 2)

        # 전체 손실: 양성 샘플 손실 + 음성 샘플 손실
        return torch.mean(pos_loss + neg_loss), cosine_sim

# 데이터셋 및 학습 코드

In [ ]:
# 3. Dataset 클래스 정의
class NewsDataset(Dataset):
    def __init__(self, news_a, news_b, feat_a, feat_b, labels):
        self.news_a = news_a
        self.news_b = news_b
        self.feat_a = feat_a
        self.feat_b = feat_b
        self.labels = labels

    def __len__(self):
        return len(self.news_a)

    def __getitem__(self, idx):
        return {
            'news_a': self.news_a[idx],
            'news_b': self.news_b[idx],
            'feat_a': torch.FloatTensor(self.feat_a[idx]),
            'feat_b': torch.FloatTensor(self.feat_b[idx]),
            'label': torch.FloatTensor([self.labels[idx]])
        }

def custom_collate(batch):
    return {
        'news_a': [item['news_a'] for item in batch],
        'news_b': [item['news_b'] for item in batch],
        'feat_a': torch.stack([item['feat_a'] for item in batch]),
        'feat_b': torch.stack([item['feat_b'] for item in batch]),
        'label': torch.stack([item['label'] for item in batch])
    }

# 4. 학습 함수
def train_model(model, dataloader, criterion, num_epochs=10, lr=1e-4):
    device = next(model.parameters()).device
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            vec_a, vec_b = model(
                batch['news_a'],
                batch['news_b'],
                batch['feat_a'].to(device),
                batch['feat_b'].to(device),
                device
            )
            loss, _ = criterion(vec_a, vec_b, batch['label'].squeeze().to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}: Loss={total_loss / len(dataloader):.4f}')
    return model

# 5. 유사 뉴스 재랭킹 함수
def find_similar_news(model, news_a, news_b, feat_a, feat_b, batch_size=32):
    device = next(model.parameters()).device
    model.eval()
    similarities = []

    for i in tqdm(range(0, len(news_a), batch_size), desc='리랭킹 중'):
        batch_news_a = news_a[i:i+batch_size]
        batch_news_b = news_b[i:i+batch_size]
        batch_feat_a = torch.FloatTensor(feat_a[i:i+batch_size]).to(device)
        batch_feat_b = torch.FloatTensor(feat_b[i:i+batch_size]).to(device)

        with torch.no_grad():
            vec_a, vec_b = model(
                batch_news_a,
                batch_news_b,
                batch_feat_a,
                batch_feat_b,
                device
            )
            batch_sim = F.cosine_similarity(vec_a, vec_b, dim=1)
            for j in range(len(batch_news_a)):
                similarities.append((i + j, batch_sim[j].item(), batch_news_a[j], batch_news_b[j]))

    return sorted(similarities, key=lambda x: x[1], reverse=True)

# 6. 실행 예시
if __name__ == '__main__':
    start = time.time()

    # 데이터 로드
    df = pd.read_csv('경로 지정')

    # 요약 없는 뉴스 제거
    df = df.dropna(subset=['news_a', 'news_b'])

    # 외부 변수 컬럼 지정
    # ext_cols_a = ['a_open', 'a_volume', 'feature1', 'something_a']
    # ext_cols_b = ['b_open', 'b_volume', 'feature2', 'something_b']
    ext_cols_a = [col for col in df.columns if 'a' in col]
    ext_cols_b = [col for col in df.columns if 'b' in col]

    # 숫자형 컬럼만 스케일링
    numeric_cols_a = [col for col in ext_cols_a if pd.api.types.is_numeric_dtype(df[col])]
    numeric_cols_b = [col for col in ext_cols_b if pd.api.types.is_numeric_dtype(df[col])]

    feat_a = df[numeric_cols_a].astype(float).values
    feat_b = df[numeric_cols_b].astype(float).values

    scaler_a = StandardScaler()
    scaler_b = StandardScaler()
    feat_a_scaled = scaler_a.fit_transform(feat_a)
    feat_b_scaled = scaler_b.fit_transform(feat_b)

    # 뉴스 텍스트 및 임시 라벨
    news_a = df['news_a'].tolist()
    news_b = df['news_b'].tolist()
    labels = df['label'].tolist() if 'label' in df.columns else [1] * len(news_a)

    # 데이터셋 및 로더
    dataset = NewsDataset(news_a, news_b, feat_a_scaled, feat_b_scaled, labels)
    dataloader = DataLoader(dataset, batch_size=8, collate_fn=custom_collate)

    # 모델 준비
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = NewsSimilarityModel(external_feature_dim=feat_a_scaled.shape[1]).to(device)
    criterion = SimilarityLoss()

    print(f'총 뉴스 쌍 개수: {len(news_a)}')

    # 학습
    model = train_model(model, dataloader, criterion, num_epochs=1, lr=1e-4)

    # 리랭킹
    results = find_similar_news(
        model=model,
        news_a=news_a,
        news_b=news_b,
        feat_a=feat_a_scaled,
        feat_b=feat_b_scaled
    )

    end = time.time()

    # 결과 출력
    print('\n리랭킹된 유사 뉴스 Top-5:')
    for i, sim, a, b in results[:5]:
        print(f'[유사도 {sim:.4f}]\n뉴스 A: {a[:80]}...\n뉴스 B: {b[:80]}...\n')

    print(f'총 {len(news_a)}쌍 | 총 소요 시간: {end - start:.2f}초')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/han-yeeun/final/db/news_2023_2025_with_stock_impact.csv'